# Train-the-Trainer Hands-on Workshop: Deep Learning Intro
Benjamin Bergner, Stefan Konigorski, Matthias Kirchler (Hasso Plattner Institut)
20. November 2020

Willkommen zum zweiten Part des Algorithmen Workshops.

## Agenda

<a name="Agenda"></a>

| Inhalt | Zeit |
| :--- | :---: |
| Einführung | 10:00 - 10:15 |
| 1) Einführung in Machine Learning mit scikit-learn | 10:15 - 11:00|
|  a) Klassische Vorhersagemodelle | |
|  b) Unüberwachtes Lernen mit Clusteranalyse & PCA | |
| Übungen in scikit-learn in Kleingruppen | 11:00 - 11:30|
| Besprechung der Ergebnisse | 11:30 - 11:45|
| Pause | 11:45 - 12:15|
| 2) Einführung in Deep Learning mit fast.ai | 12:15 - 13:00|
| Projektarbeit in fast.ai - Kleingruppen | 13:00 - 13:30|
| Besprechung der Ergebnisse | 13:30 - 13:45|
| Abschluss | 13:45 - 14:00|

## Vorbemerkung
Bevor Sie mit der Ausführung beginnen, sollten Sie den Laufzeittyp ("runtime type", falls Ihre Sprache auf Englisch eingestellt ist) auf "GPU" ändern: 

1.

<img src="https://drive.google.com/uc?export=view&id=1uTzSVK4BCAUwxJ1a1b-mB86jwZ_cbFAR" width="400"/>


2.

<img src="https://drive.google.com/uc?export=view&id=11dDC7mx-MyoNIx0jyTiiT4RLtEPmi-aO" width="400"/>


Als nächstes müssen Sie die folgende Zeile ausführen um benötigte Software zu installieren.

Die Fehlermeldung `ERROR: fastai 2.0.6 has requirement fastcore>=1.0.0, but you'll have fastcore 0.1.38 which is incompatible.` können Sie ignorieren. Sollten andere Fehler auftreten, wenden Sie sich bitte an einen der Moderatoren.

In [ ]:
# Diese Zeile muss am Anfang ein mal ausgeführt werden. Danach müssen Sie sie nicht mehr ausführen
! pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install fastai==2.0.6
! pip install fastcore==0.1.38

## fast.ai
<img style="float: left;" src="https://www.fast.ai/images/Fast.ai.png" width=250>
Für Deep Learning benötigen wir eine andere Softwarebibliothek als für klassisches Machine Learning. In fast.ai sind viele Deep Learning Algorithmen vorimplementiert und die meisten Standardoperationen sind automatisiert, so dass mithilfe von wenigen Zeilen Code, ein komplettes tiefes neuronales Netzwerk trainiert werden kann. 

In [ ]:
# Wir binden fast.ai und eine Helferfunktion so ein:
from os import path as osp
from fastai.vision.all import *

# Ein erstes tiefes neuronales Netzwerk
<hr>

## Ziel

Wir lernen die Grundlagen, wie man Bilder in fast.ai einliest und damit ein erstes neuronales Netzwerk trainiert.
<hr>

## Methoden

* Klassifikation von Bilddaten
* Convolutional Neural Networks

<hr>

## Vorbereitung: Laden von Bilddaten

In der nächsten Zelle wird ein Datensatz von verschiedenen Hunderassen automatisch heruntergeladen. Die Details sind an dieser Stelle nicht wichtig.

In [ ]:
path_to_data = 'data/imagewoof2-160'
if not osp.isdir(path_to_data):
  os.makedirs(path_to_data, exist_ok=True)
  ! wget https://www.dropbox.com/s/83f1a0lz3fwlytw/imagewoof2-160.tar -O data/imagewoof2-160.tar
  ! tar -xf data/imagewoof2-160.tar -C data

Als nächstes laden wir diesen Datensatz in einen `ImageDataLoader`, also einen Helfer, der für uns die Bilder automatisch lädt:

In [ ]:
size = 128
data = ImageDataLoaders.from_folder(
    path_to_data,
    train='train',
    valid='val',
    item_tfms=[Resize(size)],
)
data.show_batch()

## Der erste Trainingsversuch

Und jetzt können wir schon ein neuronales Netzwerk trainieren! Wir verwenden den Befehl `cnn_learner` um automatisch ein Convolutional Neural Network vorbereitet zu bekommen. An dieser Stelle verwenden wir ein sogenanntes "ResNet" der Tiefe 18.

In [ ]:
learner = cnn_learner(data, resnet18, metrics=error_rate, pretrained=False)
# der Befehl `fit` sagt dem Netzwerk, dass es trainieren soll
learner.fit(6, lr=0.1)

In [ ]:
# zeige an, wie sich das CNN geschlagen hat
learner.recorder.plot_loss()

**Was für eine Enttäuschung!** -- was haben wir falsch gemacht?

# Verbesserung 1: - Die Lernrate // Learning Rate

<img style="float: left;" src="https://miro.medium.com/max/2796/1*6Zr7nkI97IGT9e_tpgK-_g.png" width=800>

Die Lernrate beschreibt, wie große Schritte das Neuronale Netzwerk in jedem Trainingsabschnitt weitergeht. Ist die Lernrate klein, so lernt das Netzwerk ändert das Netzwerk in jedem Schritt nur sehr wenig. Ist die Lernrate groß, kann das Netzwerk auch größere Schritte tun, aber es wird auch häufig über das Ziel hinausschießen. Ein Kompromiss muss gefunden werden!

In [ ]:
# wir laden das CNN neu; sonst "erinnert" es sich an den letzen Trainingsversuch!
learner = cnn_learner(data, resnet18, metrics=error_rate, pretrained=False)

`fast.ai` ermöglicht es, die ideale Lernrate automatisch zu bestimmen!

In [ ]:
suggested_lr = learner.lr_find().lr_min
print(f'Die empfohlene Lernrate ist: {suggested_lr:.5f}')

In [ ]:
learner.fit(6, lr=suggested_lr)

In [ ]:
learner.recorder.plot_loss()

# Verbesserung 2: - Datenerweiterung // Data Augmentation
Neuronale Netzwerke funktionieren am besten, wenn sie viele Daten haben - je mehr, desto besser! Unser Datensatz ist allerdings ziemlich klein, verglichen mit anderen Deep Learning Datensätzen (ca 10 000 Bilder mit ca  1 000 Bildern pro Klasse). Idealerweise würden wir unseren Datensatz einfach vergrößern, indem wir mehr Bilder mit hinzunehmen. Das ist meist allerdings mit sehr viel Mehraufwand und Kosten verbunden.

Hier kommt die Datenerweiterung ins Spiel: anstatt viele neue Bilder zusammenzusuchen, können wir die Bilder, die wir schon haben einfach aufblähen. Ein Bild von einem Hund, das vertikalgespiegelt wurde, stellt immer noch (fast) den gleichen Hund dar - für das Neuronale Netzwerk jedoch sehen beide Bilder extrem verschieden aus!

In [ ]:
image = data.train_ds[5][0]
_, ax = subplots(1, 2)
show_image(image, ctx=ax[0], title='original')
show_image(image.flip_lr(), ctx=ax[1], title='gespiegelt')

Was können wir noch machen, außer das Bild zu spiegeln?
* kleine oder große Rotationen
* Zentrum des Bildes innerhalb des Bildes verschieben und reinzoomen
* Belichtung und Kontrast
* Perspektivveränderungen
* ...

In [ ]:
image = data.train_ds[5][0]
_, ax = subplots(1, 3)
rrc = RandomResizedCrop(128, min_scale=0.35)
show_image(image, ctx=ax[0], title='original')
show_image(rrc(image), ctx=ax[1], title='zufälliger Zoom 1')
show_image(rrc(image), ctx=ax[2], title='zufälliger Zoom 2')

Und wie binden wir das jetzt ins Training ein?
In `fast.ai` werden Data Augmentations (häufig "transforms" oder kurz "tfms" genannt) über `item_tfms` und `batch_tfms` and den Datensatz übergeben. Der Unterschied zwischen `item_tfms` und `batch_tfms` ist für uns nicht wichtig - jede Bildtransformation gehört entweder ins eine oder ins andere.


In [ ]:
size = 128
item_tfms = [
    # zufälliger Zoom
    RandomResizedCrop(size, min_scale=0.35),
    # zufällige vertikale Spiegelung
    FlipItem(0.5),
]
batch_tfms = [
    # zufällige Rotation um maximal 10°; in 50% der Bilder
    Rotate(max_deg=10, p=0.5),
    # zufällige Helligkeitsveränderung um maximal 10%; in 50% der Bilder
    Brightness(max_lighting=0.1, p=0.5)
]
data = ImageDataLoaders.from_folder(
    'data/imagewoof2-160/',
    train='train',
    valid='val',
    item_tfms=item_tfms,
    batch_tfms=batch_tfms,
)
data.show_batch()

In [ ]:
learner = cnn_learner(data, resnet18, metrics=error_rate, pretrained=False)

In [ ]:
suggested_lr = learner.lr_find().lr_min
print(f'Die empfohlene Lernrate ist: {suggested_lr:.5f}')

In [ ]:
learner.fit(6, lr=suggested_lr)

In [ ]:
learner.recorder.plot_loss()

**Immer noch weit weg von unserem Ziel!**


# Andere Ideen

**Möglichkeit 1: länger trainieren!**
    Und das funktioniert meist auch ganz gut, zu einem gewissen Grad. In unserem Beispiel etwa habe ich genau den gleichen Code mit 64 Epochen (ca 10-15 Minuten auf lokalem Server statt Google Colab) durchlaufen lassen und die Fehlerrate fällt auf etwa **27%**, eine deutliche Verbesserung. Die Datenerweiterung entfaltet ihre volle Kraft auch erst bei deutlich mehr als 10 Trainingsepochen. Aber dafür haben wir doch jetzt keine Zeit!
    
**Möglichkeit 2: kompliziertere Trainingsalgorithmen.**
    Funktioniert manchmal besser, manchmal nicht

**Möglichkeit 3: andere Netzwerkarchitektur.**
    In unserem Beispiel haben wir ein `ResNet18` verwendet, ein Standardnetzwerk mit ca 18 Layern. Wir könnten ein tieferes oder weniger tiefes Netzwerk verwenden (zB `ResNet50` mit 50 Layern), oder eine andere Art von Architektur. Da wir allerdings nur relativ wenige Datenpunkte haben, werden wir nicht viel von tieferen Architekturen profitieren, und in vielen Anwendungen sind `ResNets` der state-of-the-art, oder zumindest nah daran.
    
    
oder:

# Verbesserung 3: Transferlernen // Transfer Learning
Die Idee beim Transferlernen ist, dass Neuronale Netze aus Erfahrung lernen können. Wir können unser Netzwerk zuerst auf einem riesigen und allgemeinen Datensatz "vortrainieren" und dann das gleiche Netzwerk auf unserem kleinen, speziellen Datensatz wiederverwenden. Das tolle daran: wir müssen das Netzwerk nur ein einziges mal vortrainieren um es dann immer und immer wieder auf verschiedene Anwendungsfälle anwenden zu können. Und da das nur einmal gemacht werden muss, haben das andere Leute für uns bereits getan und wir müssen `fast.ai` einfach nur sagen, dass wir ein vortrainiertes Netzwerk haben wollen!


Der einzige Unterschied beim Laden des Netzwerkes ist das `pretrained=True` Argument!

In [ ]:
learner = cnn_learner(data, resnet18, metrics=error_rate, pretrained=True)

Beim trainieren könnten wir wieder die gleiche Funktion wie zuvor nehmen `learner.fit(12, lr=suggested_lr)`. Dies ist aber nicht optimal - um das meiste aus dem vortrainierten Netzwerk zu holen, verwenden wir die Funktion: `learner.fine_tune(freeze_epochs=4, epochs=8)`. Wir lassen an dieser Stelle auch die Lernrate aus - sie kann festgelegt werden, aber der Einfachheit halber überlassen wir das hier der `fast.ai` Bibliothek.

In [ ]:
learner.fine_tune(freeze_epochs=1, epochs=6)

**Et voilà!**

Wir schaffen es fast zu 10% Fehlerrate, während "zufälliges Raten" eine Fehlerrate von ~90% hätte! Die Unterscheidung verschiedener Hunderassen ist ein anspruchsvolles Problem, aber wir haben es geschafft, innerhalb von 45 Minuten ein durchaus akzeptables Vorhersagemodell zu trainieren!